In [2]:
from math import sqrt
import numpy as np
from numpy import concatenate
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor


from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from numpy import mean
from numpy import absolute
from numpy import loadtxt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import TransformedTargetRegressor

In [9]:
# Data Preparation
# Read dataframe
data_path = './drive/My Drive/TesiRiva/lstm.csv'

data = pd.read_csv(data_path, header=0)
to_drop = ['key (t-1)']

data = data.drop(to_drop, axis=1)
n_predictors = data.shape[1] - 1
print("Number of predictors: {} = {} days".format(n_predictors, n_predictors/4))
print(data.columns)
data_values = data.values
print(type(data_values))
print(data_values.shape)

data_values = data.values

Number of predictors: 4 = 1.0 days
Index(['delta (t-1)', 'return (t-1)', 'ttm (t-1)', 'IV (t-1)', 'IV (t)'], dtype='object')
<class 'numpy.ndarray'>
(319126, 5)


In [ ]:
data_values

In [10]:
X = data_values[:, :-1]
y = data_values[:, -1]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

myRegressor =  MLPRegressor(random_state=1, max_iter=500, verbose=1, early_stopping=True, batch_size=128)
pipeline = Pipeline(steps=[('normalize', MinMaxScaler()), ('model', myRegressor)])


In [ ]:
pipeline

In [ ]:
grid_params_pipeline = {
    'model__hidden_layer_sizes': [ (120,120), (80,80)],
    'model__activation': ['relu', 'tanh'],
    'model__batch_size': [ 512],
    'model__solver': ['sgd', 'adam'],
    'model__alpha': [0.001, 0.01],
    'model__learning_rate_init' : [0.05, 0.01],
    'model__learning_rate' : [0.05, 0.01],
    'model__learning_rate': ['adaptive', 'constant'],
}

grid = GridSearchCV(pipeline, cv=3, n_jobs=-1, param_grid=grid_params_pipeline ,scoring='r2', verbose=2)
grid.fit(train_X, train_y)
grid.score(test_X, test_y)

In [12]:
print(grid.best_estimator_)
print(grid.best_score_)

Pipeline(memory=None,
         steps=[('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('model',
                 MLPRegressor(activation='relu', alpha=0.01, batch_size=512,
                              beta_1=0.9, beta_2=0.999, early_stopping=True,
                              epsilon=1e-08, hidden_layer_sizes=(80, 80),
                              learning_rate='adaptive', learning_rate_init=0.01,
                              max_fun=15000, max_iter=500, momentum=0.9,
                              n_iter_no_change=10, nesterovs_momentum=True,
                              power_t=0.5, random_state=1, shuffle=True,
                              solver='adam', tol=0.0001,
                              validation_fraction=0.1, verbose=1,
                              warm_start=False))],
         verbose=False)
0.8825417308847828


In [13]:
import joblib

#save your model or results
joblib.dump(grid, './drive/My Drive/TesiRiva/model_file_name_last_run_minmax.pkl')

['./drive/My Drive/TesiRiva/model_file_name_last_run_minmax.pkl']

In [2]:
import joblib

best = joblib.load('./drive/My Drive/TesiRiva/model_file_name_last_run_minmax.pkl')

In [3]:
print(best.best_estimator_)
print(best.best_params_)
print(best.best_score_)

Pipeline(memory=None,
         steps=[('normalize', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('model',
                 MLPRegressor(activation='relu', alpha=0.05, batch_size=512,
                              beta_1=0.9, beta_2=0.999, early_stopping=True,
                              epsilon=1e-08, hidden_layer_sizes=(80, 80),
                              learning_rate='adaptive',
                              learning_rate_init=0.001, max_fun=15000,
                              max_iter=500, momentum=0.9, n_iter_no_change=10,
                              nesterovs_momentum=True, power_t=0.5,
                              random_state=1, shuffle=True, solver='sgd',
                              tol=0.0001, validation_fraction=0.1, verbose=1,
                              warm_start=False))],
         verbose=False)
{'model__activation': 'relu', 'model__alpha': 0.05, 'model__batch_size': 512, 'model__hidden_layer_sizes': (80, 80), 'model__learning_rate': 'ada